## Data Preperation

In [154]:
import pandas as pd
import numpy as np
import sqlite3
import time
from datetime import datetime, timedelta

In [155]:
trips = pd.read_csv('../../CSVs/trips.csv', low_memory=False)
stop_times = pd.read_csv('../../CSVs/stop_times.csv', low_memory=False)
trp_update = pd.read_csv('../../CSVs/trip_updates.csv', usecols=['oid','trip_id', 'route_id', 'timestamp', 'trip_start_time'], low_memory=False)

In [156]:
conn = sqlite3.connect('../../gtfs.db')

In [157]:
stop_times = stop_times.loc[:, ['trip_id', 'stop_id', 'stop_sequence', 'arrival_time', 'departure_time']]
stop_times.head()

,trip_id,stop_id,stop_sequence,arrival_time,departure_time
0,Logan-22-Weekday-trip,Logan-Subway,1,08:00:00,08:00:00
1,Logan-22-Weekday-trip,Logan-RentalCarCenter,2,08:04:00,08:04:00
2,Logan-22-Weekday-trip,Logan-A,3,08:09:00,08:09:00
3,Logan-22-Weekday-trip,Logan-B,4,08:12:00,08:12:00
4,Logan-22-Weekday-trip,Logan-Subway,5,08:17:00,08:17:00


In [158]:
trips = trips.loc[:, ['trip_id', 'route_id', 'service_id', 'trip_headsign']]
trips.head()

,trip_id,route_id,service_id,trip_headsign
0,Logan-22-Weekday-trip,Logan-22,Logan-Weekday,Loop
1,Logan-22-Weekend-trip,Logan-22,Logan-Weekend,Loop
2,Logan-33-Weekday-trip,Logan-33,Logan-Weekday,Loop
3,Logan-33-Weekend-trip,Logan-33,Logan-Weekend,Loop
4,Logan-55-Weekday-trip,Logan-55,Logan-Weekday,Loop


In [159]:
data_time_interval = (trp_update['timestamp'].min(),trp_update['timestamp'].max())
data_time_interval

('2017-09-25 22:33:24.000000', '2017-09-26 12:33:39.000000')

The above array shows the datetimes at which the data collection started and ended. We want to check updates on trips that we expect to run around these times. We can get this information from the static stop times table, but first we'll convert these strings to datetime objects for easier manipulation.

In [160]:
start = datetime.strptime(data_time_interval[0][11:19], '%H:%M:%S').time()
end = datetime.strptime(data_time_interval[1][11:19], '%H:%M:%S').time()
start, end

(datetime.time(22, 33, 24), datetime.time(12, 33, 39))

In [161]:
stop_times = stop_times[stop_times['arrival_time'] < '24:00:00' ]
static_arrival_times = stop_times['arrival_time'].apply(lambda x: datetime.strptime(x, '%H:%M:%S').time())

The stop_times dataframe now only contains trips that have static arrival times that fall below hour 24. This is because the python datetime.time object cannot handle hours past 24. It is possible perhaps to use the timedelta python object to handle this edge case. For now those trips will be taken out. The arrival times have been converted into datetime.time objects and will be used for indexing.

In [162]:
stop_times = stop_times[(static_arrival_times < end) | (static_arrival_times > start)]
stop_times.head()

,trip_id,stop_id,stop_sequence,arrival_time,departure_time
0,Logan-22-Weekday-trip,Logan-Subway,1,08:00:00,08:00:00
1,Logan-22-Weekday-trip,Logan-RentalCarCenter,2,08:04:00,08:04:00
2,Logan-22-Weekday-trip,Logan-A,3,08:09:00,08:09:00
3,Logan-22-Weekday-trip,Logan-B,4,08:12:00,08:12:00
4,Logan-22-Weekday-trip,Logan-Subway,5,08:17:00,08:17:00


In [165]:
# sanity check
static_arrival_times = stop_times['arrival_time'].apply(lambda x: datetime.strptime(x, '%H:%M:%S').time())
stop_times[(static_arrival_times > end) & (static_arrival_times < start)]

,trip_id,stop_id,stop_sequence,arrival_time,departure_time


The stop_times dataframe only has static arrival times that fall under the period in which data was collected. Meaning that some of the trip ids in the dataframe could have corresponding rows in the trip updates table.

## Trip Updates

In [166]:
temp_series = trp_update['timestamp'].copy()
trp_update['timestamp'] = trp_update['timestamp'].apply(lambda x : datetime.strptime(x[11:19], '%H:%M:%S').time())

In [167]:
# function to calculate the difference of two datetime.time objects
time_diff = lambda x,y,z: datetime.combine(z, x) -  datetime.combine(z, y)

In [168]:
first_stops = stop_times[stop_times['stop_sequence'] == 1].loc[:, ['trip_id', 'stop_id', 'arrival_time']]
first_stops = first_stops.set_index('trip_id')

The first_stops dataframe contains the expected arrival time for the first stop of each trip. This will be used later to try and retreive updates of trips from the trip updates table that are expected to have been running during the data collection period. The set index command is helpful because it allows for indexing by trip_id in order to get an expected arrival time. Note that this is doable because the first_stops dataframe does not have repeated trip_ids. Hopefully all first stops are those that are labeled 1 under stop_sequence for each trip.

In [169]:
trp_update.head()

,oid,trip_id,route_id,trip_start_time,timestamp
0,1,35242107,1,NaN,22:33:24
1,2,35242108,1,NaN,22:33:24
2,3,35242109,1,NaN,22:33:24
3,4,35242110,1,NaN,22:33:24
4,5,35242111,1,NaN,22:33:24


In [179]:
trp_update = trp_update[trp_update['trip_id'].isin(first_stops.index.values)]

The above code minimizes the amount of records that need to be searched through. The set of trip_ids that are expected to be within the data collection period have been identified through static GTFS data, therefore, the trip updates associated with trip_ids that are not in this set are not important.

In [227]:
# this is just used in order to be able to do arithmetic with timestamps
dummy = datetime(1,1,1,0,0,0)
hour_23 = datetime(1,1,1,23,59,59)

for idx, row in trp_update.iterrows():
    trip_id = row['trip_id']
    timestamp = row['timestamp']

    try:
        first_stop_arrival_time = first_stops.loc[trip_id, 'arrival_time']
    except:
        continue
    first_stop_arrival_time = datetime.strptime(first_stop_arrival_time, '%H:%M:%S').time()

    start = min(timestamp, first_stop_arrival_time)
    end = max(timestamp, first_stop_arrival_time)

    t_diff = time_diff(end, start, dummy)

    if (t_diff < timedelta(seconds=1800)):
        print('yeet')
        break


The above code is supposed to find live trip updates. In other words trip updates that were trips that were running during the data collection period